# Milestone 4 partial solution:  Improve the machine learning model
 
## Objective
 
- Iteratively improve the model accuracy: use external datasets, work
  on feature extraction, and tune hyper-parameters.
 
## Requirements
 
Install the Python package that we have created in our previous milestone, which contains some of the useful functions,

In [1]:
! pip install --upgrade https://public-sym.s3-eu-west-1.amazonaws.com/LP-safety-US-cities/safety-events-cities-1.0.0.tar.gz

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for safety-events-cities: filename=safety_events_cities-1.0.0-py3-none-any.whl size=2974 sha256=fd5baae842c890a6e18a67e7c983c4c689b19ef7afcca9e24c6140662646b210
  Stored in directory: /home/assitan/.cache/pip/wheels/31/3b/3a/02c5de71c1358088b9a2e6d6c1793ea657f179a14d3a6ec554
Successfully built safety-events-cities
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


This solution also requires the `holidays` package,

In [2]:
!pip install holidays

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 172 kB 6.0 MB/s            
     |████████████████████████████████| 47 kB 15.9 MB/s            
     |████████████████████████████████| 5.4 MB 16.9 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for pymeeus: filename=PyMeeus-0.5.11-py3-none-any.whl size=730984 sha256=5120d8720a78c20d0974e7861b671c5ef07d2c3bd6f8400e5b92457bf6ab044a
  Stored in directory: /home/assitan/.cache/pip/wheels/a0/8b/b2/810ae5a6f970c8be4725353400d643c90de1c0f023a9884ee7
Successfully built pymeeus
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
# this plugin requires to run "pip install nb-black"
%load_ext nb_black

# Reload modules automatically when importing code
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [4]:
import pandas as pd

import matplotlib.pyplot as plt  # noqa

<IPython.core.display.Javascript object>

We start by list most frequent categories, for future reference,

In [7]:
x = pd.read_parquet("data/safety-SanFrancisco-2.parquet")

<IPython.core.display.Javascript object>

We use the `load_data` function created in the previous milestone, that filters the data to 2016-2019, select one category, and count the number of events per neighborhood for each day.

In [11]:
from safety_events_cities.milestone3 import load_data

df = load_data(
    "../data/safety-SanFrancisco-2.parquet",
    category="Street and Sidewalk Cleaning",
    years=[2016, 2017, 2018, 2019],
    freq="6H",
)

df.head()

,dateTime,neighborhood,count
0,2016-01-01 00:00:00,Mission,2
1,2016-01-01 00:00:00,Nob Hill,1
2,2016-01-01 06:00:00,Ingleside,1
3,2016-01-01 06:00:00,Mission,9
4,2016-01-01 06:00:00,Nob Hill,5


<IPython.core.display.Javascript object>

## 1-2. Using external data

To enrich the features available for modeling, we will add two externals dataset, in particular holiday and weather data.

###  1.1 Taking into account holidays

Federal US holidays as well as local state holidays for California can be obtained with the 
[holidays](https://pypi.org/project/holidays/) package

In [8]:
from datetime import date
import holidays

<IPython.core.display.Javascript object>

We create a new boolean column to indicate whether a given date is a holiday,

To approximately evaluate the impact of the added variable, we can compare the mean number of events per day on a holiday and a non holiday day. There are 7-25% fewer events of the considered type during holidays.

Similarly we create an new boolean variable for dates corresponding to holiday eves.

### 1.2 Weather data

We next load the [weather data](https://docs.opendata.aws/noaa-ghcn-pds/readme.html) from the NOAA Global Historical Climatology Network. The data is already with daily periodicity, however the values for different sensors are encoded row wise.

We use `set_index` followed by `unstack` pandas methods, to create a pivot table where each field has a separate column. Another important step is that we need to fill missing values, using either previous or following values.

For instance, let's visualize the mean daily temperature,

We create a function to merge the weather information with the original data,

To measure the relevance of added features on the target variable, we can compute the Pearson correlation between them.

There is no strong linear correlation between weather and the number of events that we are trying to predict. However a non-linear model model, as used in the 

## 3. Gradient boosting model

Gradient boosting decision trees (GBDT) is accurate and effective off-the-shelf algorithm for regression or classification that works well that is suitable to tabular data. Here we will use the histogram based variation included in scikit-learn (`HistGradientBoostingRegressor`) that is inspired by [LightGBM](https://lightgbm.readthedocs.io/en/latest/).

We re-create similar preprocessing pipeline to that of step 2 except for using OrdinalEncoder instead of OneHotEncoder, as tree based models are not suitable for very high dimensional features.

To more reliably evaluate the model performance we can also compute these metrics with time based cross-validation.

## 4. Hyper-parameter search

For finding optimal hyper-parameters, we are interested both in better test
scores but also in reducing overfitting to make the model more robust. The
latter is particularly important given that the training set it small with only
a few 1000s samples and we can easily overfit.  Because there are numerous
hyper-parameters to test, we have sampled them randomly with
`RandomizedSearchCV` estimator.

We observe that adding L2 regularisation has a
similar effect to reducing the maximum depth. The Poisson loss also provides
very competitive results which is consistent with it being suitable for
modeling count and frequency data.

It's also useful to visually compare the predicted values with the observed values for different neighborhoods,

We reproduce the weekly periodicity and trends, however there is a significant
day to day variance that still isn't accounted for by the model.

## 4. Other attempted approaches

Following approaches were attempted but did not yield a measurable improvement in results,
- applying a transformation to the target variable with [`TransformedTargetRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.TransformedTargetRegressor.html) to make its distribution closer to the Normal distribution. Note that when using `HistGradientBoostingRegressor(loss="poisson")` a [log link](https://en.wikipedia.org/wiki/Generalized_linear_model#Link_function) is used internally.
- weight more recent observations higher during training using exponentially decreasing `sample_weight`.

## Conclusion

We have explored several approaches for improving the predictions, randing from using external datasets to searching optimal hyper-parameters. To explore, we could also try a purely time series based approach with for instance the [fb-prothet](https://facebook.github.io/prophet/) package.

In the following milestone we will see try to make the current model more interpretable and evaluate the possibility of bias.